# Data Analysis using Python

## Analysis of Udacity student data

We're working with 3 data files containing information on enrollments, engagement of students with Udacity and submission of projects.

Having a quick browse of the data files and column descriptions I came up with these 5 questions to answer.
1. Which students most likely to cancel? Which factors have the most impact
2. Which students most likely to stay? Which factors have the most impact
3. Ideal time to spend for distinction project rating?
4. Test account vs other account for days to cancel
5. Any patterns for join/cancel dates?


Here is a [link](not yet implemented) to the description of columns which is necessary to understand what questions we should ask about the data.

In [194]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#suppress displaying long numbers in scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

#Tells Jupyter to print results of any variable on a new line so tables will look good
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Files to load
enrollments_file = 'enrollments.csv'
engagement_file = 'daily_engagement.csv'
submissions_file = 'project_submissions.csv'

#Read all 3 files into respective dataframes
dfenr = pd.read_csv(enrollments_file)
dfeng = pd.read_csv(engagement_file)
dfsub = pd.read_csv(submissions_file)

Exploring the data set

In [2]:
#Find unique number of students and total rows
enrollment_num_rows = len(dfenr['account_key'])
enrollment_num_unique_students = len(dfenr['account_key'].unique())

#Rename the column to account key so it's uniform in all 3 data frames. Inplace=True replaces the orginal with this df
dfeng.rename(columns={'acct':'account_key'},inplace=True)
engagement_num_rows = len(dfeng['account_key'])
engagement_num_unique_students = len(dfeng['account_key'].unique())

#Find unique number of students and total rows
submission_num_rows = len(dfsub['account_key'])
submission_num_unique_students = len(dfsub['account_key'].unique())

#To diplay our results nicely. Create a data frame with named indices
data1 = [["Total Rows","Unique Rows"],
             [enrollment_num_rows,enrollment_num_unique_students],
             [engagement_num_rows,engagement_num_unique_students],
             [submission_num_rows,submission_num_unique_students]]

dfdisplay1 = pd.DataFrame(data1[1:], index = ['Enrollment','Engagement','Submission'])
dfdisplay1.columns = data1[0]
print("\nUnique students vs Total Rows")
dfdisplay1



Unique students vs Total Rows


,Total Rows,Unique Rows
Enrollment,1640,1302
Engagement,136240,1237
Submission,3642,743


We see that Unique students for Enrollment and Engagement is off, this should not be the case as all students should be in the Engagement table even if their engagement times are zero.
<br>Begin by taking a look at values of Unique students enrolled  that have no engagement time. 
<br>If we assume the structure (Enrollment:Engagement), we'll take a left join where Enrollment and Engagement don't overlap

In [3]:
#Going to use dfexpl for explore to have a look.
#Take the Left Join for Enrollments and Engagements first
dfexpl = pd.merge(dfenr, dfeng, on='account_key', how='left')
#Display where num_courses_visited is null, so there are no Engagement values for these particular students.
dfexpl[dfexpl.num_courses_visited.isnull()]


,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
14704,1219,canceled,2014-11-12,2014-11-12,0.0,False,True,NaN,NaN,NaN,NaN,NaN
16161,871,canceled,2014-11-13,2014-11-13,0.0,False,True,NaN,NaN,NaN,NaN,NaN
38732,1218,canceled,2014-11-15,2014-11-15,0.0,False,True,NaN,NaN,NaN,NaN,NaN
41520,654,canceled,2014-12-04,2014-12-04,0.0,False,True,NaN,NaN,NaN,NaN,NaN
41521,654,canceled,2014-12-04,2014-12-04,0.0,False,True,NaN,NaN,NaN,NaN,NaN
41522,654,canceled,2014-12-03,2014-12-03,0.0,False,True,NaN,NaN,NaN,NaN,NaN
41620,964,canceled,2014-12-04,2014-12-04,0.0,False,True,NaN,NaN,NaN,NaN,NaN
51339,817,canceled,2014-12-09,2014-12-09,0.0,False,True,NaN,NaN,NaN,NaN,NaN
52190,1171,canceled,2015-01-07,2015-01-07,0.0,False,True,NaN,NaN,NaN,NaN,NaN
52983,884,canceled,2015-01-08,2015-01-08,0.0,False,True,NaN,NaN,NaN,NaN,NaN


What we find is that the join_date and cancel_date are on the same day for the majority of students. So they could have signed up and cancelled before the Engagement table was populated. This leaves them with a days_to_cancel of zero. This may or may not be an issue depending on what analysis we would like to perform, but finding the reason for this anomoly is a win, and we might use this information later.

We continue and see if there are any more anomolies that don't match our above finding

In [4]:
#Found students that still had a status of "current" but no engagement information
dfexpl[(dfexpl.num_courses_visited.isnull()) & (dfexpl.days_to_cancel != 0)]

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
79912,1304,canceled,2015-01-10,2015-03-10,59.0,True,True,NaN,NaN,NaN,NaN,NaN
79913,1304,canceled,2015-03-10,2015-06-17,99.0,True,True,NaN,NaN,NaN,NaN,NaN
177737,1101,current,2015-02-25,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaN


In the above code we find 3 students with no engagement information, and days_to_cancel not equal to zero. The commonality here is that they are all Udacity Test Accounts. This seen from the is_udacity variable, which is True for test accounts. We'll now remove test accounts from all our data and proceed.

In [5]:
#Remove all test accounts from data. NT for No Test accounts
dfenrNT = dfenr[dfenr.is_udacity == False].copy()
len(dfenrNT)
len(dfenr)

1622

1640

We move to the exploration phase. We ask the following question:
<br>How do numbers in the daily engagement table differ for students who pass the first project?

This raises the following problems if we don't filter our data:
1. Data after the project submission will be included
2. This could also mean we're observing studnet engagement for courses after the first project
3. Data of different lengths of time are being compared

Hence as a first filter we'll remove any students that have cancelled Udacity memberships as well as students who have stayed enrolled more than 7 days. As there is a 7 day free trial period. That will leave us with paid students.

In [6]:
#Turn off warnings first
#pd.options.mode.chained_assignment = 'warn' 

#NT is no test accounts, P is for paid, U is for unique
#Use .copy() at the end to prevent settingwithcopywarning
dfenrNTP = dfenrNT[(dfenrNT.days_to_cancel.isnull()) | (dfenrNT.days_to_cancel > 7)].copy()

#sorts first by account key then by join date, then only keeps the max join_date
dfenrNTPU = dfenrNTP.sort_values(['account_key','join_date']).drop_duplicates(['account_key'],keep='last')

#how many entries we have now
len(dfenrNTPU)

#take a look at our entries
dfenrNTPU.head()


995

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
464,0,current,2015-01-09,NaN,NaN,False,False
1233,1,current,2015-05-06,NaN,NaN,False,False
1612,2,current,2015-06-08,NaN,NaN,False,False
308,3,current,2015-03-10,NaN,NaN,False,False
1487,4,current,2015-05-12,NaN,NaN,False,False


Now we create a data set with only students (paid students with no test accounts) who have engaged in courses within the first week joining.

In [7]:
#Take the Left Join for Enrollments and Engagements
dfenrNTPUeng = pd.merge(dfenrNTPU, dfeng, on='account_key', how='left')

#convert date columns to type datetime for easier manipulation
dfenrNTPUeng['join_date'] = pd.to_datetime(dfenrNTPUeng['join_date'])
dfenrNTPUeng['utc_date'] = pd.to_datetime(dfenrNTPUeng['utc_date'])
#dfenrNTPUeng.dtypes
#Define students who have engaged within 1 week with 0<= engagement <7
dfenrNTPUeng1w = dfenrNTPUeng[((dfenrNTPUeng['utc_date'] - dfenrNTPUeng['join_date']).dt.days < 7) & ((dfenrNTPUeng['utc_date'] - dfenrNTPUeng['join_date']).dt.days >= 0)].copy()

#Number of entries returned. Without first sorting before dropping duplicates we get an incorrect value
len(dfenrNTPUeng1w)

#a snippet of our data
dfenrNTPUeng1w

6919

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,current,2015-01-09,NaN,NaN,False,False,2015-01-09,1.0,11.679374,0.0,0.0
1,0,current,2015-01-09,NaN,NaN,False,False,2015-01-10,2.0,37.284887,0.0,0.0
2,0,current,2015-01-09,NaN,NaN,False,False,2015-01-11,2.0,53.633746,0.0,0.0
3,0,current,2015-01-09,NaN,NaN,False,False,2015-01-12,1.0,33.489270,0.0,0.0
4,0,current,2015-01-09,NaN,NaN,False,False,2015-01-13,1.0,64.779678,0.0,0.0
5,0,current,2015-01-09,NaN,NaN,False,False,2015-01-14,1.0,151.617577,1.0,0.0
6,0,current,2015-01-09,NaN,NaN,False,False,2015-01-15,1.0,142.395964,0.0,0.0
230,1,current,2015-05-06,NaN,NaN,False,False,2015-05-06,1.0,9.640248,0.0,0.0
231,1,current,2015-05-06,NaN,NaN,False,False,2015-05-07,0.0,0.000000,0.0,0.0
232,1,current,2015-05-06,NaN,NaN,False,False,2015-05-08,0.0,0.000000,0.0,0.0


Out of curiosity we want to know the Average minutes spent in the classroom for these students who have engaged within the first week
To do this we'll average the total number of minutes each students has engaged.

In [8]:
#First groupby account key, then look at descriptive stats for total_minutes_visited
dfenrNTPUeng1w.groupby(['account_key'])['total_minutes_visited'].describe().head()

#Finding the totals for each student, we'll sum each group and output the descriptive stats we want
#w for the 1 week set
wsumcol = dfenrNTPUeng1w.groupby(['account_key'])['total_minutes_visited'].sum()
wsum = wsumcol.sum()
wmean = wsumcol.mean()
#pandas stdev uses Bessel's correction by default N-1, use ddof=0 for population N
wstd = wsumcol.std(ddof=0)
wmin = wsumcol.min()
wmax = wsumcol.max()

print("Sum: {}\nMean: {}\nStandard Deviation: {}\nMin: {}\nMax: {}".format(wsum,wmean,wstd,wmin,wmax))

,count,mean,std,min,25%,50%,75%,max
account_key,,,,,,,,
0,7.0,70.697214,54.770605,11.679374,35.387079,53.633746,103.587821,151.617577
1,7.0,2.653769,4.536729,0.000000,0.000000,0.000000,4.468068,9.640248
2,7.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,7.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,7.0,4.760201,12.594307,0.000000,0.000000,0.000000,0.000000,33.321405


Sum: 305174.7851196613
Mean: 306.7083267534284
Standard Deviation: 412.99693340852923
Min: 0.0
Max: 3564.7332644989997


The maximum time spent on Udacity courses in a week was around 60hrs, which is huge, but plausible. We'll check the student's account to see if the result makes sense.

In [9]:
#find the student with the maximum total mins
wsumcol[wsumcol==wmax]

#observe their study times and dates
dfenrNTPUeng1w[dfenrNTPUeng1w.account_key == 163]

account_key
163    3564.733264
Name: total_minutes_visited, dtype: float64

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
27698,163,current,2015-07-09,NaN,NaN,False,False,2015-07-09,4.0,850.519340,4.0,0.0
27699,163,current,2015-07-09,NaN,NaN,False,False,2015-07-10,6.0,872.633923,6.0,0.0
27700,163,current,2015-07-09,NaN,NaN,False,False,2015-07-11,2.0,777.018904,6.0,0.0
27701,163,current,2015-07-09,NaN,NaN,False,False,2015-07-12,1.0,294.568774,2.0,0.0
27702,163,current,2015-07-09,NaN,NaN,False,False,2015-07-13,3.0,471.213978,1.0,0.0
27703,163,current,2015-07-09,NaN,NaN,False,False,2015-07-14,2.0,298.778345,1.0,0.0
27704,163,current,2015-07-09,NaN,NaN,False,False,2015-07-15,0.0,0.000000,0.0,0.0


Now we'll use the same method to find the number of lessons completed instead of time spent.

In [10]:
#First groupby account key, then look at descriptive stats for total_minutes_visited
dfenrNTPUeng1w.groupby(['account_key'])['lessons_completed'].describe().head()

#Finding the totals for each student, we'll sum each group and output the descriptive stats we want
#w for the 1 week set
wLsumcol = dfenrNTPUeng1w.groupby(['account_key'])['lessons_completed'].sum()
wLsum = wLsumcol.sum()
wLmean = wLsumcol.mean()
#pandas stdev uses Bessel's correction by default N-1, use ddof=0 for population N
wLstd = wLsumcol.std(ddof=0)
wLmin = wLsumcol.min()
wLmax = wLsumcol.max()

print("Sum: {}\nMean: {}\nStandard Deviation: {}\nMin: {}\nMax: {}".format(wLsum,wLmean,wLstd,wLmin,wLmax))

,count,mean,std,min,25%,50%,75%,max
account_key,,,,,,,,
0,7.0,0.142857,0.377964,0.0,0.0,0.0,0.0,1.0
1,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


Sum: 1628.0
Mean: 1.636180904522613
Standard Deviation: 3.0025612998294333
Min: 0.0
Max: 36.0


Next we'll use the num_courses_visited column to analyse the number of days students visited a classroom.

In [11]:
#First groupby account key, then look at descriptive stats for total_minutes_visited
dfenrNTPUeng1w.groupby(['account_key'])['num_courses_visited'].describe().head()

#list(dfenrNTPUeng1w.groupby(['account_key'])['num_courses_visited'])

#Create function to count only non-zero values
def nonzerocount(group):
    nzc = 0
    for item in group:
        if item != 0:
            nzc += 1
    return nzc

#Use .apply to pass the groups to my new function
wNCcol = dfenrNTPUeng1w.groupby(['account_key'])['num_courses_visited'].apply(nonzerocount)
wNCsum = wNCcol.sum()
wNCmean = wNCcol.mean()
#pandas stdev uses Bessel's correction by default N-1, use ddof=0 for population N
wNCstd = wNCcol.std(ddof=0)
wNCmin = wNCcol.min()
wNCmax = wNCcol.max()

print("Sum: {}\nMean: {}\nStandard Deviation: {}\nMin: {}\nMax: {}".format(wNCsum,wNCmean,wNCstd,wNCmin,wNCmax))

,count,mean,std,min,25%,50%,75%,max
account_key,,,,,,,,
0,7.0,1.285714,0.487950,1.0,1.0,1.0,1.5,2.0
1,7.0,0.285714,0.487950,0.0,0.0,0.0,0.5,1.0
2,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,7.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,7.0,0.142857,0.377964,0.0,0.0,0.0,0.0,1.0


Sum: 2853
Mean: 2.8673366834170855
Standard Deviation: 2.2551980029196876
Min: 0
Max: 7


Continuing off the same data set, we'll now find the number of students that pass a certain project and the number that don't pass this project. The project is to do with the New York Subway.

Subway proj lesson key '746169184' or '3176718735'

In [167]:
# We'll need to use data from our submissions file as this has lesson keys and records for passing or not
# We'll use a left join so we don't create any unwanted null values
dfenrNTPUeng1wsub = pd.merge(dfenrNTPUeng1w, dfsub, on='account_key', how='left')
#dfenrNTPUeng1wsub.head()

#Find totals then how many passed and didn't pass
#Note that students who passed either "PASSED" or get "DISTINCTION"
oldKeydf = dfenrNTPUeng1wsub[dfenrNTPUeng1wsub.lesson_key == 746169184]
newKeydf = dfenrNTPUeng1wsub[dfenrNTPUeng1wsub.lesson_key == 3176718735]

#combine dataframes so we have a complete set of all of the subway project, ignore index true allows the new index to be in sequential
subwaydf = pd.concat([oldKeydf,newKeydf],ignore_index=True)
subwaydfPass = subwaydf[((subwaydf.lesson_key == 3176718735) | (subwaydf.lesson_key == 746169184)) & ((subwaydf.assigned_rating == 'PASSED') | (subwaydf.assigned_rating == 'DISTINCTION'))]
subwaystudents = len(subwaydf['account_key'].unique())
subwaystudentsPass = len(subwaydfPass['account_key'].unique())
subwaystudentsNonPass = subwaystudents - subwaystudentsPass

print("Students who passed this project: {}\nStudents who did not pass this project: {}".format(subwaystudentsPass,subwaystudentsNonPass))

Students who passed this project: 647
Students who did not pass this project: 33


Next we think of some questions to ask about passing students and non passing students. 

In [168]:
#Here we implement a (left join where B is null) which could possibly be called a (NOT right join).
#build data of nonpassing students, step one, an intermediate df with an indicator
subwaydfInd = pd.merge(subwaydf,subwaydfPass,indicator=True,how='left') 

# Then selecting only non-passing students
subwaydfNonPass = subwaydfInd[subwaydfInd['_merge'] == 'left_only']
subwaydfNonPass

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,creation_date,completion_date,assigned_rating,lesson_key,processing_state,_merge
0,542,current,2015-07-10,NaN,nan,False,False,2015-07-10,4.00,248.75,4.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
1,542,current,2015-07-10,NaN,nan,False,False,2015-07-11,0.00,0.00,0.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
2,542,current,2015-07-10,NaN,nan,False,False,2015-07-12,3.00,153.61,0.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
3,542,current,2015-07-10,NaN,nan,False,False,2015-07-13,1.00,56.21,0.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
4,542,current,2015-07-10,NaN,nan,False,False,2015-07-14,1.00,162.16,0.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
5,542,current,2015-07-10,NaN,nan,False,False,2015-07-15,1.00,102.89,1.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
6,542,current,2015-07-10,NaN,nan,False,False,2015-07-16,1.00,313.33,1.00,0.00,2015-08-17,2015-08-18,INCOMPLETE,746169184.00,EVALUATED,left_only
7,0,current,2015-01-09,NaN,nan,False,False,2015-01-09,1.00,11.68,0.00,0.00,2015-04-03,2015-04-03,INCOMPLETE,3176718735.00,EVALUATED,left_only
9,0,current,2015-01-09,NaN,nan,False,False,2015-01-09,1.00,11.68,0.00,0.00,2015-03-09,2015-03-11,INCOMPLETE,3176718735.00,EVALUATED,left_only
10,0,current,2015-01-09,NaN,nan,False,False,2015-01-09,1.00,11.68,0.00,0.00,2015-03-13,2015-03-25,INCOMPLETE,3176718735.00,EVALUATED,left_only


Questions. Which of the following affects passing or failing a project:
1. Time spent (total_minutes_visited)
2. Lessons completed
3. Projects completed
4. Project ID (lesson_key)

To describe our findings we'll use mean, standard deviation, minimum and maximum values.

In [193]:
#Passing statistics for all students
subPtimeCol = subwaydfPass.groupby(['account_key'])['total_minutes_visited']
subPtimeMean = subPtimeCol.sum().mean()
subPtimeStd = subPtimeCol.sum().std(ddof=1)
subPtimeMin = subPtimeCol.sum().min()
subPtimeMax = subPtimeCol.sum().max()

#Non-passing statistics for all students
subNPtimeCol = subwaydfNonPass.groupby(['account_key'])['total_minutes_visited']
subNPtimeMean = subNPtimeCol.sum().mean()
subNPtimeStd = subNPtimeCol.sum().std(ddof=1)
subNPtimeMin = subNPtimeCol.sum().min()
subNPtimeMax = subNPtimeCol.sum().max()

#To diplay our results nicely. Create a table
data2 = [[subPtimeMean,subNPtimeMean],
         [subPtimeStd,subNPtimeStd],
         [subPtimeMin,subNPtimeMin],
         [subPtimeMax,subNPtimeMax]]

print("\nTime Spent by Passing and Non-passing students in hours")

#Convert to hours
# data2a = pd.DataFrame(data2).divide(60).round(2)
data2a = pd.DataFrame(data2).round(2)

#Make our row and column labels nice
data2a.rename(columns={0:'Passing Students',1:'Non-passing students'}, index={0:'Mean',1:'Standard Deviation',2:'Minimum',3:'Maximum'},inplace=True)

#I want to see the percentage difference
data2a['% change Non-pass'] = 100*((data2a['Non-passing students'] - data2a['Passing Students'])/data2a['Non-passing students']).round(2).fillna(0)
data2a


Time Spent by Passing and Non-passing students in hours


,Passing Students,Non-passing students,% change Non-pass
Mean,402.23,681.50,41.00
Standard Deviation,472.31,887.54,47.00
Minimum,0.00,0.00,0.00
Maximum,3564.73,7129.47,50.00


Here we find students that didn't pass spent on average 41% more time on projects.

Next we look at lessons completed.

In [174]:
#Passing statistics for all students
subPtimeCol = subwaydfPass.groupby(['account_key'])['lessons_completed']
subPtimeMean = subPtimeCol.sum().mean()
subPtimeStd = subPtimeCol.sum().std(ddof=1)
subPtimeMin = subPtimeCol.sum().min()
subPtimeMax = subPtimeCol.sum().max()

#Non-passing statistics for all students
subNPtimeCol = subwaydfNonPass.groupby(['account_key'])['lessons_completed']
subNPtimeMean = subNPtimeCol.sum().mean()
subNPtimeStd = subNPtimeCol.sum().std(ddof=1)
subNPtimeMin = subNPtimeCol.sum().min()
subNPtimeMax = subNPtimeCol.sum().max()

#To diplay our results nicely. Create a table
data2 = [[subPtimeMean,subNPtimeMean],
         [subPtimeStd,subNPtimeStd],
         [subPtimeMin,subNPtimeMin],
         [subPtimeMax,subNPtimeMax]]

print("\nLessons completed by Passing and Non-passing students")

#Convert to dataframe
data2a = pd.DataFrame(data2).round(2)

#Make our row and column labels nice
data2a.rename(columns={0:'Passing Students',1:'Non-passing students'}, index={0:'Mean',1:'Standard Deviation',2:'Minimum',3:'Maximum'},inplace=True)

#I want to see the percentage difference
data2a['% change Non-pass'] = 100*((data2a['Non-passing students'] - data2a['Passing Students'])/data2a['Non-passing students']).round(4).fillna(0)
data2a

2.091190108191654

Here Non-passing students complete more lessons on average. 

Next we'll look at projects completed

In [171]:
#Passing statistics for all students
subPtimeCol = subwaydfPass.groupby(['account_key'])['projects_completed']
subPtimeMean = subPtimeCol.sum().mean()
subPtimeStd = subPtimeCol.sum().std(ddof=1)
subPtimeMin = subPtimeCol.sum().min()
subPtimeMax = subPtimeCol.sum().max()

#Non-passing statistics for all students
subNPtimeCol = subwaydfNonPass.groupby(['account_key'])['projects_completed']
subNPtimeMean = subNPtimeCol.sum().mean()
subNPtimeStd = subNPtimeCol.sum().std(ddof=1)
subNPtimeMin = subNPtimeCol.sum().min()
subNPtimeMax = subNPtimeCol.sum().max()

#To diplay our results nicely. Create a table
data2 = [[subPtimeMean,subNPtimeMean],
         [subPtimeStd,subNPtimeStd],
         [subPtimeMin,subNPtimeMin],
         [subPtimeMax,subNPtimeMax]]

print("\nProjects completed by Passing and Non-passing students")

#Convert to dataframe
data2a = pd.DataFrame(data2).round(5)

#Make our row and column labels nice
data2a.rename(columns={0:'Passing Students',1:'Non-passing students'}, index={0:'Mean',1:'Standard Deviation',2:'Minimum',3:'Maximum'},inplace=True)

#I want to see the percentage difference
data2a['% change Non-pass'] = 100*((data2a['Non-passing students'] - data2a['Passing Students'])/data2a['Non-passing students']).round(4).fillna(0)
data2a


Projects completed by Passing and Non-passing students


,Passing Students,Non-passing students,% change Non-pass
Mean,0.01,0.00,-122.84
Standard Deviation,0.11,0.09,-21.61
Minimum,0.00,0.00,0.00
Maximum,2.00,2.00,0.00


We find most students don't complete a project with a range of 0 to 2 and a mean very close to zero. Since the numbers are so low. I had a look at both Passing and Non-passing students.

In [172]:
print('\nProjects completed by Non-passing students')
subwaydfNonPass[['account_key','projects_completed']][subwaydfNonPass.projects_completed >0]

print('\nProjects completed by Passing students')
subwaydfPass[['account_key','projects_completed']][subwaydfPass.projects_completed >0]



Projects completed by Non-passing students


,account_key,projects_completed
1589,108,2.00



Projects completed by Passing students


,account_key,projects_completed
1588,108,2.00
3640,240,1.00
6251,401,1.00
8317,525,1.00
8681,550,1.00


I'm curious to see student 108's data.

In [173]:
#We check in the indicator dataframe
subwaydfInd[subwaydfInd.account_key == 108].sort_values(by=['completion_date'],ascending=[True])

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,creation_date,completion_date,assigned_rating,lesson_key,processing_state,_merge
1583,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-20,2.00,25.91,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1585,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-21,1.00,3.41,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1587,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-22,2.00,99.12,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1589,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-23,1.00,35.83,0.00,2.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1591,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-24,0.00,0.00,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1593,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-25,0.00,0.00,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1595,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-26,0.00,0.00,0.00,0.00,2015-02-18,2015-02-23,INCOMPLETE,3176718735.00,EVALUATED,left_only
1582,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-20,2.00,25.91,0.00,0.00,2015-03-21,2015-03-24,PASSED,3176718735.00,EVALUATED,both
1584,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-21,1.00,3.41,0.00,0.00,2015-03-21,2015-03-24,PASSED,3176718735.00,EVALUATED,both
1586,108,canceled,2015-04-20,2015-05-14,24.00,False,True,2015-04-22,2.00,99.12,0.00,0.00,2015-03-21,2015-03-24,PASSED,3176718735.00,EVALUATED,both


Student 108 completes 2 projects on 2015-02-23 and 2 projects on 2015-03-24. This is plausible as we don't have the information for which projects were completed. We only know that the subway project was completed on 2015-03-24 and incomplete before. This is only an interesting aside.